In [5]:
from ib_async import *
import time

util.startLoop()  # uncomment this line when in a notebook

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=1)

ib.reqMarketDataType(1)  # Use free, delayed, frozen data
contract = Forex('EURUSD')
bars = ib.reqHistoricalData(
    contract, endDateTime='', durationStr='1 D',
    barSizeSetting='1 min', whatToShow='MIDPOINT', useRTH=True)

# convert to pandas dataframe (pandas needs to be installed):
df = util.df(bars)
df.sort_values(by=['date'], ascending=False, inplace=True)

display(df)

# Make sure to disconnect the client once finished with client work
ib.disconnect()
time.sleep(1)

"""
# Define the stock contract
contract = Stock('AAPL', 'SMART', 'USD')

# Define order parameters
quantity = 10
entry_price = 150  # Your entry price
take_profit_price = 160  # Target price
stop_loss_price = 145  # Stop loss price

# Create the bracket order
bracket = ib.bracketOrder(
    action='BUY',
    quantity=quantity,
    limitPrice=entry_price,
    takeProfitPrice=take_profit_price,
    stopLossPrice=stop_loss_price
)

# Place the bracket order
for order in bracket:
    ib.placeOrder(contract, order)

print("Bracket order placed successfully!")

summary = ib.accountSummary()
"""





,date,open,high,low,close,volume,average,barCount
1424,2025-05-09 16:59:00-04:00,1.124855,1.125305,1.124750,1.125300,-1.0,-1.0,-1
1423,2025-05-09 16:58:00-04:00,1.124795,1.125030,1.124795,1.124855,-1.0,-1.0,-1
1422,2025-05-09 16:57:00-04:00,1.125105,1.125130,1.124780,1.124795,-1.0,-1.0,-1
1421,2025-05-09 16:56:00-04:00,1.124875,1.125180,1.124850,1.125105,-1.0,-1.0,-1
1420,2025-05-09 16:55:00-04:00,1.124785,1.124910,1.124760,1.124875,-1.0,-1.0,-1
...,...,...,...,...,...,...,...,...
4,2025-05-08 17:19:00-04:00,1.122525,1.122525,1.122525,1.122525,-1.0,-1.0,-1
3,2025-05-08 17:18:00-04:00,1.122525,1.122525,1.122525,1.122525,-1.0,-1.0,-1
2,2025-05-08 17:17:00-04:00,1.122525,1.122525,1.122525,1.122525,-1.0,-1.0,-1
1,2025-05-08 17:16:00-04:00,1.122525,1.122525,1.122525,1.122525,-1.0,-1.0,-1


'\n# Define the stock contract\ncontract = Stock(\'AAPL\', \'SMART\', \'USD\')\n\n# Define order parameters\nquantity = 10\nentry_price = 150  # Your entry price\ntake_profit_price = 160  # Target price\nstop_loss_price = 145  # Stop loss price\n\n# Create the bracket order\nbracket = ib.bracketOrder(\n    action=\'BUY\',\n    quantity=quantity,\n    limitPrice=entry_price,\n    takeProfitPrice=take_profit_price,\n    stopLossPrice=stop_loss_price\n)\n\n# Place the bracket order\nfor order in bracket:\n    ib.placeOrder(contract, order)\n\nprint("Bracket order placed successfully!")\n'

In [39]:
for i in summary:
    # print(i[1], i[2])
    if i[1] == "AvailableFunds":
        print(i[2])


1000000.00
